In [ ]:
import torch
import torchvision.models as models
import os
from torch import nn

In [ ]:
def print_model_size(mdl):
    torch.save(mdl.state_dict(), "tmp.pt")
    print("%.2f MB" %(os.path.getsize("tmp.pt")/1e6))
    os.remove('tmp.pt')

In [ ]:
class MobileNet(torch.nn.Module):
    def __init__(self):
        super(MobileNet, self).__init__()
        self.model = models.MobileNetV2()
        
        
        for param in self.model.parameters():
            param.requires_grad = False
            
        
        self.model.classifier[1] = nn.Sequential(
            nn.Linear(in_features=self.model.classifier[1].in_features,out_features=512),
            nn.LeakyReLU(negative_slope=0.02,inplace=True),
            nn.BatchNorm1d(num_features=512),
            nn.Dropout(p=0.4,inplace=True),
            nn.Linear(in_features=512,out_features=2),
            nn.Softmax(dim=1))
        
        # print(self.model)

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
model = MobileNet()

In [ ]:
print_model_size(model)

In [ ]:
import platform
chip = platform.processor()

if chip == 'arm':
    backend = 'qnnpack'
elif chip in ['x86_64', 'i386']:
    backend = 'fbgemm'
else:
    raise SystemError("Backend is not supported")

print(f"Using {backend} backend engine for {chip} CPU")

torch.backends.quantized.engine = backend

In [ ]:
from torch.quantization.quantize_fx import prepare_fx, convert_fx

example_inputs = (torch.randn(1, 3, 28, 28),)
qconfig = {
    "": torch.quantization.get_default_qconfig(backend),
    "module_name": [
        ("features.13", None),    
        ("features.14", None),
        ("features.15", None),
        ("features.16", None),
        ("features.17", None),
    ]
}
model_prepared = prepare_fx(model.eval(), qconfig,example_inputs)


In [ ]:
from DataLoader import prepare_data_loaders

In [ ]:
data_loader, data_loader_test = prepare_data_loaders()

In [ ]:
def train_epoch(model, criterion, optimizer, data_loader, device):
    model.train()
    
    epoch_loss = 0.0
    num_batches = len(data_loader)
    
    for batch_idx, (image, target) in enumerate(data_loader):
        image, target = image.to(device), target.to(device)
        
        output = model(image)
        
        loss = criterion(output, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_epoch_loss = epoch_loss / num_batches
    print(f"Epoch Training Loss: {avg_epoch_loss:.4f}")
    

In [ ]:
def evaluate(model, criterion, data_loader, device):
    
    epoch_loss = 0.0
    
    correct_predictions = 0
    total_predictions = 0
    
    num_batches = len(data_loader)
    
    with torch.no_grad():
       
        for image, target in data_loader:
            image, target = image.to(device), target.to(device)
            output = model(image)
            loss = criterion(output, target)

            epoch_loss += loss.item()
            
            _, predicted = torch.max(output, 1)  # Get the predicted class index
            correct_predictions += (predicted == target).sum().item()
            total_predictions += target.size(0)
            
    avg_epoch_loss = epoch_loss / num_batches
    print(f"Epoch Test Loss: {avg_epoch_loss:.4f}")
    
    accuracy = correct_predictions / total_predictions
    print(f"Epoch Accuracy: {accuracy:.4f}")
    

In [ ]:
criterion = nn.CrossEntropyLoss(reduction='mean')
optimizer = torch.optim.AdamW(model_prepared.parameters(), lr = 0.001)

In [ ]:
import copy

In [ ]:

for nepoch in range(10):
    train_epoch(model_prepared, criterion, optimizer, data_loader, torch.device('cpu'))
    

    # Check the accuracy after each epoch
    model_quantized = copy.deepcopy(model_prepared)
    dynamic_resnet = convert_fx(model_quantized)
    dynamic_resnet.eval()
    evaluate(dynamic_resnet,criterion, data_loader_test,torch.device('cpu'))
